In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import tensorflow_io as tfio
import os
import pydub

In [ ]:
#Dataset preprocessing
def load_wav_16k_mono(filename):
    a = pydub.AudioSegment.from_mp3(filename)
    y = np.array(a.get_array_of_samples())
    return y


def preprocess(file_path):
    wav = load_wav_16k_mono("data/train/" + file_path)
    wav = wav.astype(np.float32, order='C')
    spectrogram = tf.signal.stft(wav, frame_length=160, frame_step=128)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    #print(spectrogram.shape)
    spectrogram = np.pad(np.asarray(spectrogram), ((0,2500-spectrogram.shape[0]), (0,0), (0, 0)), 'constant')
    return spectrogram



X = []
Y = []

n = 0
n_0 = 0
with open("train_gt.csv", 'r') as file:
    for line in file:
        audio_file, label = line.split(',')
        if (int(label) == 0 and n_0 <= 3234) or int(label) == 1:
            X.append(preprocess(audio_file))
            Y.append(int(label))
            if (int(label) == 0):
                n_0 += 1
            print(n)
            n += 1


In [ ]:
X

In [26]:
a = 0
b = 0
for i in Y:
    if i == 1:
        a += 1
    else:
        b += 1
print(a, b)

3234 3234


In [25]:
del X[1]
del Y[1]

In [27]:
Y = np.array(Y)
X = np.stack(X, axis=0)

In [28]:
np.save('Train.npy', X)
np.save('TrainY.npy', Y)

In [2]:
X = np.load('Train.npy')
Y = np.load('TrainY.npy')

In [30]:
X.shape

(6468, 2500, 129, 1)

In [31]:
Y.shape

(6468,)

In [57]:
model = tf.keras.Sequential([

])
model.compile('Adam', loss='BinaryCrossentropy', metrics=["accuracy"])

In [ ]:
hystory = model.fit(X[:5100], Y[:5100], epochs=20)

In [ ]:
plt.title('Loss')
plt.plot(hystory.history['loss'], 'r')
plt.show()
plt.title('Accuracy')
plt.plot(hystory.history['accuracy'], 'r')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(X[5100:], Y[5100:], verbose=2)

print('Test accuracy:', test_acc)

In [ ]:
predictions = model.predict(X[5100:])

In [ ]:
predicted = np.array([1 if x[0] >= 0.5 else 0 for x in predictions.tolist()])
actual = np.array(Y[5100:])
conf_mat = confusion_matrix(actual, predicted)

displ = ConfusionMatrixDisplay(confusion_matrix=conf_mat)
displ.plot()

In [ ]:
model.summary()

In [ ]:
model.save('audio_model2')